## **CharacterTextSplitter**
- CharacterTextSplitter는 가장 기본적인 텍스트 분할기입니다. <br>

- 특정 구분자(separator)를 기준으로 텍스트를 나누고, 각 덩어리의 크기(chunk size)를 문자의 개수로 제한합니다.

#### **특징:**

- **분할 방식:** 단일 문자를 구분자로 사용해 텍스트를 분할합니다. 기본값은 공백( ) 또는 개행 문자(\n).

- **기준:** chunk_size는 <u>**문자 수**</u>를 기준으로 합니다.

- **장점:** 매우 간단하고 직관적입니다.

- **단점:** <u>의미를 고려하지 않고 단순히 문자 수에 따라 자르기 때문에, 문장이나 단어의 중간을 끊어버릴 수 있어 문맥이 손실될 위험이 큽니다.</u> 예를 들어, "안녕하세요. 제 이름은 김지수입니다."라는 문장을 문자 수 기준으로 자르면 "안녕하세요. 제 이름은"과 " 김지수입니다."처럼 의미 없는 조각이 생길 수 있습니다.

## **RecursiveCharacterTextSplitter**

- RecursiveCharacterTextSplitter는 가장 널리 사용되는 분할기입니다.

- 여러 개의 구분자를 재귀적으로 적용하여 텍스트를 나눕니다. 

- 의미 있는 경계를 우선적으로 찾기 때문에 CharacterTextSplitter의 단점을 보완합니다.

#### **특징:**

- **분할 방식**: 구분자 목록(separators)을 정의된 순서대로 시도하며 텍스트를 분할합니다.

    - 기본 순서는 ["\n\n", "\n", " ", ""]입니다.

    - 먼저 문단(\n\n) 단위로 나누고, 다음 기준으로 줄 단위(\n), 단어 단위( ), 마지막으로 문자 단위( )로 자릅니다.

- **기준:** chunk_size는 <u>**문자 수**</u>를 기준으로 합니다.

- **장점:** <u>문장, 단락 등 의미 있는 단위로 텍스트를 유지하려 노력하기 때문에 문맥을 보존하는 데 효과적입니다.</u>

- **단점:** 복잡한 구조의 문서에서는 완벽하게 의미를 보존하지 못할 수도 있습니다.

## **TokenTextSplitter**
TokenTextSplitter는 텍스트를 문자가 아닌 <u>**토큰(token) 단위**</u>로 분할합니다. 토큰은 일반적으로 언어 모델이 텍스트를 처리하는 최소 단위로, 단어, 부분 단어(sub-word), 구두점 등이 될 수 있습니다. <u>이는 LLM이 정해진 최대 토큰 수를 넘지 않도록 할 때 유용</u>합니다.

#### **특징:**

- **분할 방식:** Hugging Face의 transformers 라이브러리 같은 특정 토크나이저를 사용해 텍스트를 토큰으로 변환한 후, 토큰 수를 기준으로 분할합니다.

- **기준:** chunk_size는 <u>토큰 수를 기준</u>으로 합니다.

- **장점:** <u>LLM의 토큰 제한(예: GPT-4의 8192 토큰)에 맞춰 정확하게 텍스트를 분할</u>할 수 있습니다. 이는 RAG(검색 증강 생성) 시스템에서 맥락 창(context window)을 효율적으로 활용하는 데 필수적입니다.

- **단점:** 토크나이저를 사용하기 때문에 다른 분할기보다 설정이 복잡하고, <u>토크나이저에 따라 분할 결과가 달라질 수 있습니다</u>.

## **SemanticChunker**
<br>

``` python
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.
text_splitter = SemanticChunker(OpenAIEmbeddings())
```
<br>

SemanticChunker는 기존의 규칙 기반 분할기(예: 문자 수, 토큰 수, 특정 구분자)와 달리, 텍스트의 **의미적 내용(semantic content)**을 기준으로 문서를 분할하는 고급 텍스트 분할기입니다.

이 분할기는 문맥과 토픽이 갑자기 바뀌는 지점(의미적 불연속성)을 찾아내어 덩어리(chunk)를 나눕니다. 결과적으로, 하나의 덩어리 안에 의미적으로 매우 응집된 내용만 포함하게 되어 RAG(검색 증강 생성)와 같은 LLM 애플리케이션의 성능을 크게 향상시킬 수 있습니다.

작동 원리
SemanticChunker는 **임베딩 모델(embedding model)**을 사용하여 텍스트의 의미를 분석합니다.

작은 단위로 분할: 먼저, 긴 문서를 문장이나 작은 문단과 같은 매우 작은 단위로 나눕니다.

임베딩 생성: 각 작은 단위에 대해 임베딩 모델을 사용하여 의미를 나타내는 벡터(vector)를 생성합니다.

유사도 측정: 인접한 두 단위의 임베딩 벡터 간 **코사인 유사도(cosine similarity)**를 계산합니다. 유사도가 높으면 두 단위가 비슷한 내용을 다룬다는 뜻이고, 낮으면 토픽이 바뀌었다는 의미입니다.

의미적 분할: 유사도가 사전에 설정된 임계값(threshold)보다 낮아지는 지점을 찾아내고, 바로 그 지점에서 덩어리를 나눕니다.

이 과정은 마치 문맥의 흐름을 그래프로 시각화하여 토픽이 급격하게 변하는 계곡 지점을 찾아내는 것과 유사합니다.